# Sam controlnet evaluation
After training our custom controlnet, we evaluate its performance empirically and qualitatively

## Imports

In [1]:
import os

if os.environ.get('KAGGLE_KERNEL_RUN_TYPE', ''):  
    !pip install -U git+https://github.com/huggingface/accelerate.git
    !pip install -q huggingface-hub==0.14.1
    !pip install -q datasets==2.12.0
    !pip install -q diffusers==0.17.1
    !pip install -q xformers
    !pip install -U git+https://github.com/huggingface/transformers.git


!pip install torchvision --no-cache --force-reinstall --index-url https://download.pytorch.org/whl/cu118

  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-2urdgjk2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-2urdgjk2
  Resolved https://github.com/huggingface/accelerate.git to commit e60a4243988a636bda8a6bf99044fb313d5a9e0e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-wq7_sy_t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-wq7_sy_t
  Resolved https://github.com/huggingface/transformers.git to commit 0b7b4429c78de68acaf72224eb6dae43616d820c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━

## Import utils from dataset notebook

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import gc
import PIL
import io
from accelerate import PartialState

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    del mask
    gc.collect()

plt.ioff()

def get_mask_img(masks, size, random_color=True):
    masks = np.array(masks)
    
    fig = plt.figure(facecolor='black')    
    ax = plt.gca()

    ax.set_autoscale_on(True)
    for mask in masks:
        show_mask(mask, ax=ax, random_color=random_color)
    plt.axis("off")
    
    fig.canvas.draw()
#     plt.tight_layout(pad=0)
    buf = io.BytesIO()
    

    fig.savefig(buf, format="jpg", bbox_inches='tight', pad_inches=0)
    buf.seek(0)
    img = PIL.Image.open(buf)
#     img = PIL.Image.frombytes('RGB', fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
    plt.close('all')
    plt.clf()
    plt.cla()
    fig.canvas.flush_events()
    del mask
    gc.collect() 
    img = img.resize(size)
    
    return img

def img2masks(img, random_color=True):
    print('generating')
    outputs = generator(img, points_per_batch=64)
    print('get mask 2 img')
    return get_mask_img(outputs["masks"], img.size)



/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Load models

In [4]:
from transformers import pipeline
import torch

generator = pipeline("mask-generation", model="facebook/sam-vit-base", device=0)

In [5]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch
from datasets import load_dataset
from diffusers import UniPCMultistepScheduler

controlnet_original = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny")
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet_original
)



pipe.to("cuda")
pipe.safety_checker = lambda images, clip_input: (images, [False])

dataset = load_dataset("baptistecolle/sam-controlnet-final-test")  # load dataset from "baptistecolle/sam-controlnet-final-test" cf hugginface doc

controlnet_sam_trained = ControlNetModel.from_pretrained("karoldobiczek/sam-controlnet")
pipe_sam = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet_sam_trained
)


pipe_sam.scheduler = UniPCMultistepScheduler.from_config(pipe_sam.scheduler.config)


pipe_sam.to("cuda")
pipe_sam.safety_checker = lambda images, clip_input: (images, [False])

# controlnet_sam_trained = timm.create_model('hf_hub:karoldobiczek/sam-controlnet', pretrained=True) # our controlnet ask karol directly for url of is controlnet model


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/baptistecolle___parquet/baptistecolle--sam-controlnet-final-test-3904b6930f3b2a10/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


## Evaluation
We compute IoC for on our test dataset

In [57]:
def get_IoU(img_1, img_2):
    px1 = list(img_1.getdata())
    px2 = list(img_2.getdata())
    width1, height1 = img_1.size
    width2, height2 = img_2.size
    px1 = [px1[i * width1:(i + 1) * width1] for i in range(height1)]
    px2 = [px2[i * width2:(i + 1) * width2] for i in range(height2)]
    
    one_two = {}
    two_one = {}
    count = 0
    
    one_two[(0,0,0)] = (0,0,0)
    two_one[(0,0,0)] = (0,0,0)
    for i in range(len(px1)):
        for j in range(len(px1[i])):
            p1 = px1[i][j]
            p2 = px2[i][j]
            if (not p1 in one_two) and (not p2 in two_one):
                one_two[p1] = p2
                two_one[p2] = p1

            if ((p1 in one_two) and (p2 in two_one) and (one_two[p1] == p2) and (two_one[p2] == p1)) or (p2 == (0,0,0)):
                count += 1
    
    return count / (len(px1)*len(px1[0]))

In [58]:
# display/produced 6 images
IoU_original = []
IoU_trained = []

i=0
for example in dataset['train']:
    %cd /kaggle/working
    original_image = example["image"] # look at "baptistecolle/sam-controlnet-final" for sam segmentation mask
    condition_image = example["conditioning_image"] # look at "baptistecolle/sam-controlnet-final" for sam segmentation mask
    caption = example["text"] # look at "baptistecolle/sam-controlnet-final" for sam segmentation mask
    
    print("caption", caption)


    %mkdir result${i}
    output_diffuser_original = pipe(
        caption,
        condition_image,

    )
    output_diffuser_original.images[0].show()
    %cd /tmp
    %mv *.PNG /kaggle/working/result${i}/original.PNG
    

    
    print("here2")
    print("here3")

    
    # same with our trained controlnet             
    output_diffuser_sam_trained = pipe_sam(
        caption,
        condition_image,
    )
    
    print("done")
    
    output_diffuser_sam_trained.images[0].show()
    %mv *.PNG /kaggle/working/result${i}/trained.PNG

    
    print("doing sam")
    
    segmented_image_original = img2masks(output_diffuser_original.images[0]) 
    segmented_image_original.show()
    %mv *.PNG /kaggle/working/result${i}/original_mask.PNG
    segmented_image_sam_trained = img2masks(output_diffuser_sam_trained.images[0]) 
    print('showing')
    segmented_image_sam_trained.show()
    %mv *.PNG /kaggle/working/result${i}/trained_mask.PNG
    # get interesection over union IoU
    IoU_original.append(get_IoU(segmented_image_original, condition_image)) 
    IoU_trained.append(get_IoU(segmented_image_sam_trained, condition_image))                                
           
    print(IoU_original[-1])
    print(IoU_trained[-1])
    print("Original ControlNet average IoU: " + str(sum(IoU_original) / len(IoU_original)))
    print("Trained ControlNet average IoU: "  + str(sum(IoU_trained)  / len(IoU_trained)))
    i += 1
# now displayed 10 original_image with therir respective output_diffuser_original output_diffuser_sam_trained, segmented_image_original, segmented_image_sam_trained
# for empircal visualisation for blog post,
                                            
print("Original ControlNet average IoU: " + str(sum(IoU_original) / len(IoU_original)))
print("Trained ControlNet average IoU: "  + str(sum(IoU_trained)  / len(IoU_trained)))


/kaggle/working
caption A man with a red helmet on a small moped on a dirt road. 


  0%|          | 0/50 [00:00<?, ?it/s]

/tmp


/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpx5fkyog1.PNG'


here2
here3


...

generating


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


get mask 2 img
showing


/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmppr_akc4d.PNG'


0.3944401041666667
0.4077647569444444
Original ControlNet average IoU: 0.33248768278706137
Trained ControlNet average IoU: 0.3748904130591844
Original ControlNet average IoU: 0.33248768278706137
Trained ControlNet average IoU: 0.3748904130591844


In [ ]:
%ls /tmp
%cd /tmp
%mv *.PNG /kaggle/working

In [ ]:
%cd /kaggle/working
%ls

In [42]:
%ls

__notebook_source__.ipynb  imgs/  result0/  state.db


In [53]:
%cd /kaggle/working


/kaggle/working


In [54]:
%ls

__notebook_source__.ipynb  result0/  result2/  result4/  state.db
imgs/                      result1/  result3/  result5/


In [55]:
%rm -fr result*

In [56]:
%cd /tmp
%rm *.PNG

/tmp
rm: cannot remove '*.PNG': No such file or directory


In [1]:
%ls

__notebook_source__.ipynb  result133/  result170/  result28/  result65/
imgs/                      result134/  result171/  result29/  result66/
result0/                   result135/  result172/  result3/   result67/
result1/                   result136/  result173/  result30/  result68/
result10/                  result137/  result174/  result31/  result69/
result100/                 result138/  result175/  result32/  result7/
result101/                 result139/  result176/  result33/  result70/
result102/                 result14/   result177/  result34/  result71/
result103/                 result140/  result178/  result35/  result72/
result104/                 result141/  result179/  result36/  result73/
result105/                 result142/  result18/   result37/  result74/
result106/                 result143/  result180/  result38/  result75/
result107/                 result144/  result181/  result39/  result76/
result108/                 result145/  result182/  result4/   res

In [2]:
%mkdir eval
%mv result* eval
%ls

__notebook_source__.ipynb  eval/  imgs/  state.db


In [10]:
!zip -r evaluation.zip eval

  adding: eval/ (stored 0%)
  adding: eval/result166/ (stored 0%)
  adding: eval/result166/trained.PNG (deflated 0%)
  adding: eval/result166/original_mask.PNG (deflated 1%)
  adding: eval/result166/original.PNG (deflated 0%)
  adding: eval/result166/trained_mask.PNG (deflated 0%)
  adding: eval/result160/ (stored 0%)
  adding: eval/result160/trained.PNG (deflated 0%)
  adding: eval/result160/original_mask.PNG (deflated 0%)
  adding: eval/result160/original.PNG (deflated 0%)
  adding: eval/result160/trained_mask.PNG (deflated 0%)
  adding: eval/result21/ (stored 0%)
  adding: eval/result21/trained.PNG (deflated 0%)
  adding: eval/result21/original_mask.PNG (deflated 1%)
  adding: eval/result21/original.PNG (deflated 0%)
  adding: eval/result21/trained_mask.PNG (deflated 0%)
  adding: eval/result37/ (stored 0%)
  adding: eval/result37/trained.PNG (deflated 0%)
  adding: eval/result37/original_mask.PNG (deflated 2%)
  adding: eval/result37/original.PNG (deflated 0%)
  adding: eval/result

In [11]:
%ls

__notebook_source__.ipynb  eval/  eval.zip  evaluation.zip  imgs/  state.db
